### Retrieval Augmented Generation (RAG)

### budowa VectorStore (FAISS) i Retrievera

In [ ]:
from langchain_community.vectorstores import FAISS

emb = OpenAIEmbeddings()  # wymaga OPENAI_API_KEY
vectorstore = FAISS.from_texts(splitted, embedding=emb)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

query = "Po co używa się retrievera?"
context = retriever.get_relevant_documents(query)
print("Znaleziony kontekst:")
for i, c in enumerate(context, 1):
    print(f"{i}.", c.page_content)

### prosty łańcuch RAG (prompt + kontekst + LLM)

In [ ]:
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "Udziel precyzyjnej odpowiedzi wyłącznie na podstawie KONTEKSTU. Jeśli brak danych — powiedz, że nie wiesz."),
    ("system", "KONTEKST:\n{context}"),
    ("user", "{question}")
])

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke("Czym jest FAISS i do czego służy?"))

### Przykład RAG

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Dokumenty źródłowe
docs = [
    "LangChain to framework do pracy z LLM.",
    "RAG łączy retrieval kontekstu z generacją odpowiedzi.",
    "FAISS to biblioteka do przechowywania i wyszukiwania embeddingów."
]

# Split
splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
splits = splitter.create_documents(docs)

# Embeddings + vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(splits, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# Prompt RAG
prompt = ChatPromptTemplate.from_messages([
    ("system", "Odpowiadaj tylko na podstawie kontekstu:\n{context}"),
    ("user", "{question}")
])

def format_docs(docs):
    return "\n".join(d.page_content for d in docs)

# Pipeline
rag_chain = (
    {"context": retriever | format_docs, "question": lambda x: x["question"]}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke({"question": "Co to jest FAISS?"}))


### Pipeline RAG

In [ ]:
from langchain_community.document_loaders import TextLoader

# Ładowanie dokumentu
with open("sample.txt", "w") as f:
    f.write("LangGraph pozwala budować grafy stanów. Retrieval-Augmented Generation łączy search i generation.")

loader = TextLoader("sample.txt")
docs = loader.load()

# Split
splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=20)
splits = splitter.split_documents(docs)

# Embeddings + FAISS
vectorstore = FAISS.from_documents(splits, embedding=embeddings)
retriever = vectorstore.as_retriever()

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "Odpowiadaj tylko na podstawie kontekstu:\n{context}"),
    ("user", "{question}")
])

rag_chain = (
    {"context": retriever | format_docs, "question": lambda x: x["question"]}
    | prompt
    | llm
    | StrOutputParser()
)

print(rag_chain.invoke({"question": "Co to jest LangGraph?"}))
